**Additional Resources - Power BI Commands.**

[https://learn.microsoft.com/en-us/powershell/power-bi/overview?view=powerbi-ps](https://learn.microsoft.com/en-us/powershell/power-bi/overview?view=powerbi-ps)

**Installing and checking prerequisite**

In [ ]:
# What version of PowerShell is install 5.1 or greater
$PSVersionTable

In [ ]:
# Install Nuget Packages - Admin rights required.
Install-PackageProvider -Name NuGet -Force

In [ ]:
# Install Power BI Commands - Admin rights required
Install-Module -Name MicrosoftPowerBIMgmt -Force

In [ ]:
# Verify Power BI Commands are installed. 
# Get-Command -Module MicrosoftPowerBIMgmt
(Get-Command Get-PowerBIWorkspace).dll
Get-Command | Where-Object { $_.Source -match 'MicrosoftPowerBIMgmt.Workspaces'}

**Login to Power BI Service**

In [ ]:
# Saved Power BI Login
$authUrl = 'https://login.microsoftonline.com/98--------------43a5ccb4/oauth2/token';
$clientId="682-----------------8a243ec3"
$clientSecret = "hHX-------------9WI20oyI"

In [ ]:
# Manual Login to Power BI - Use Email & AD Password
#Connect-PowerBIServiceAccount -Credential (Get-Credential)
Login-PowerBI

In [ ]:
# Manually Sign out of Power BI
Disconnect-PowerBIServiceAccount

**Workspaces**

In [ ]:
# Show All Active Workspaces on PowerBI
$workspaces = Get-PowerBIWorkspace -Scope Organization -Include All -All | Where-Object { $_.State -match 'Active' } | Select-Object Id, Name, Type
$workspaces

In [ ]:
# Delete Workspace by Name
$SearchName = "Workspace Name"
 (Get-PowerBIWorkspace -Name $SearchName) | foreach {
         $Id = $_.Id
         $Url = "groups/$Id"
         Invoke-PowerBIRestMethod -Url $Url -Method Delete }

In [24]:
# Find Deleted Workspaces by Name
$SearchName = "Workspace Name"
$workspaces2 = Get-PowerBIWorkspace -Scope Organization -Include All -All | Where-Object { $_.State -match 'Deleted' -and $_.Name -eq $SearchName }
$workspaces2

In [ ]:
# Restore Workspace by Workspace Id
$workspaceid = "20e6b0e4-e237-462f-8b25-caec664550da"
$workspaceRestoreName = "Workspace Name restored"
$assignOwner = "someone@xyz.com"
Restore-PowerBIWorkspace -Id $workspaceid -RestoredName $workspaceRestoreName  -AdminEmailAddress $assignOwner

**Power BI Reports**

In [ ]:
# Download PBXI files 
$reportid = "6112d4f0-a0b1-4a42-a1dd-59e920df95b8"
$saveto = "C:\source\"
$filename = "powerbi.pbxi"
$url = "https://api.powerbi.com/v1.0/myorg/reports/" + $reportid + "/Export"
$download = $saveto+$filename
Invoke-WebRequest $url -OutFile $download

In [ ]:
#Log in to Power BI Service
Login–PowerBI  –Environment Public 	

#First, Collect all (or one) of the workspaces in a parameter called PBIWorkspace
$PBIWorkspace = Get-PowerBIWorkspace 							# Collect all workspaces you have access to
#$PBIWorkspace = Get-PowerBIWorkspace -Name 'My Workspace Name' 	# Use the -Name parameter to limit to one workspace

#Now collect todays date
$TodaysDate = Get-Date –Format "yyyyMMdd" 

#Almost finished: Build the outputpath. This Outputpath creates a news map, based on todays date
$OutPutPath = "C:\Source\" + $TodaysDate 

#Now loop through the workspaces, hence the ForEach
ForEach($Workspace in $PBIWorkspace)
{
	#For all workspaces there is a new Folder destination: Outputpath + Workspacename
	$Folder = $OutPutPath + "\" + $Workspace.name 
	#If the folder doens't exists, it will be created.
	If(!(Test-Path $Folder))
	{
		New-Item –ItemType Directory –Force –Path $Folder
	}
	#At this point, there is a folder structure with a folder for all your workspaces 
	
	
	#Collect all (or one) of the reports from one or all workspaces 
	$PBIReports = Get-PowerBIReport –WorkspaceId $Workspace.Id 						 # Collect all reports from the workspace we selected.
	#$PBIReports = Get-PowerBIReport -WorkspaceId $Workspace.Id -Name "My Report Name" # Use the -Name parameter to limit to one report
		
		#Now loop through these reports: 
		ForEach($Report in $PBIReports)
		{
			#Your PowerShell comandline will say Downloading Workspacename Reportname
			Write-Host "Downloading "$Workspace.name":" $Report.name 
			
			#The final collection including folder structure + file name is created.
			$OutputFile = $OutPutPath + "\" + $Workspace.name + "\" + $Report.name + ".pbix"
			
			# If the file exists, delete it first; otherwise, the Export-PowerBIReport will fail.
			 if (Test-Path $OutputFile)
				{
					Remove-Item $OutputFile
				}
			
			#The pbix is now really getting downloaded
			Export-PowerBIReport –WorkspaceId $Workspace.ID –Id $Report.ID –OutFile $OutputFile
		}
}

In [ ]:
Get-PowerBIReport -Scope Organization

**Power BI Datasets**

**Power BI Users**

In [ ]:
# All Users assigned to Workspaces (Uncomment to Export to CSV File)
Get-PowerBIWorkspace -Scope Organization -Include All -All  | 
ForEach-Object {
$Workspace = $_.name
$WorkspaceId = $_.Id
foreach ($User in $_.Users) {
[PSCustomObject]@{
Workspace = $Workspace
WorkspaceId = $WorkspaceId
User = $User.accessright    
Identifier   =$user.Identifier}}} #| Export-CSV "C:\source\WorkspaceDetails.csv" -NoTypeInformation